# 모델 작업

## sbert모델을 ONNX모델로 export

In [1]:
from pathlib import Path
import transformers
from transformers.convert_graph_to_onnx import convert

In [3]:
convert(framework="pt", model="jhgan/ko-sroberta-multitask", output=Path("./model/onnx/ko-sroberta-multitask.onnx"), opset=11)

ONNX opset version set to: 11
Loading pipeline (model: jhgan/ko-sroberta-multitask, tokenizer: jhgan/ko-sroberta-multitask)
Creating folder model\onnx
Using framework PyTorch: 1.13.1+cpu
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


## 양자화

In [5]:
#!pip install onnxruntime

     ---------------------------------------- 6.5/6.5 MB 21.9 MB/s eta 0:00:00
     ---------------------------------------- 46.0/46.0 kB ? eta 0:00:00
     ---------------------------------------- 6.5/6.5 MB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB ? eta 0:00:00
     -------------------------------------- 532.6/532.6 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 95.2/95.2 kB ? eta 0:00:00


In [8]:
from onnxruntime.quantization import quantize_dynamic, QuantType

In [9]:
quantize_dynamic("./model/onnx/ko-sroberta-multitask.onnx", "./model/onnx/ko-sroberta-multitask_uint8.onnx", weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.5/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/

# 모델3 불러오기

In [1]:
from onnxruntime import InferenceSession
from sentence_transformers import SentenceTransformer, util
import torch

In [2]:
embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")
embedder_onnx = InferenceSession("./model/onnx/ko-sroberta-multitask.onnx", providers=["CPUExecutionProvider"])
embedder_onnx_uint8 = InferenceSession("./model/onnx/ko-sroberta-multitask_uint8.onnx", providers=["CPUExecutionProvider"])

## 풀링함수
SentenceTransformer모델을 자동으로 풀링을 해서 임베딩  
ONNX모델은 BERT와 비슷하므로 둘의 임베딩 값을 맞추려면 ONNX모델 결과의 풀링을 거처야함

In [3]:
def mean_pooling(model_output, attention_mask):
    model_output = torch.from_numpy(model_output[0])
    # model_output의 첫번째 요소는 모든 token의 embedding을 포함
    token_embeddings = model_output
    attention_mask = torch.from_numpy(attention_mask)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask, input_mask_expanded, sum_mask

# 데이터 불러오기

In [4]:
from tqdm import tqdm
import pandas as pd
from glob import glob

In [5]:
files = sorted(glob('./data/*.csv'))
data_fit = pd.DataFrame()
for file in tqdm(files[:1]):
    df_day = pd.read_csv(file, usecols=["okt"])
    data_fit = pd.concat([data_fit, df_day])
data_fit.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.50it/s]


# 모델 테스트

## 기초모델

In [150]:
%%time
embeddings = embedder.encode(data_fit.iloc[:,0], show_progress_bar=True)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

CPU times: total: 11min 21s
Wall time: 2min 54s


In [153]:
%%time
embeddings2 = embedder.encode(data_fit.iloc[:,0], show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

CPU times: total: 11min 30s
Wall time: 2min 53s


In [203]:
arc_content = """2011년 한국을 떠나 러시아로 귀화한 전 쇼트트랙 선수 빅토르 안(38·한국명 안현수)의 국내 복귀 시도가 일단 무산됐다. 경기도 성남시청 직장운동부 빙상팀 코치에 지원했으나, 최종 후보에 들지 못해서다.

성남시는 29일 “시청 빙상팀 코치직 채용 전형에 빅토르 안을 포함해 7명이 지원했는데, 빅토르 안은 상위 2배수 후보에 들지 않았다”고 밝혔다. 시 관계자는 “서류와 면접 심사를 통해 기술, 소통 능력 등 여러 요소를 종합해 판단했다”며 “빙상계 여론과 언론 보도 등을 통해 나오는 시각도 평가에 반영됐다”고 했다.

성남시는 지난해 12월 19일 빙상팀 코치를 뽑기 위한 채용 공고를 냈다. 빅토르 안과 베이징 동계올림픽에서 중국 대표팀을 이끈 김선태 전 감독 등 7명이 지원했다. 김 전 감독도 최종 후보에 들지 못한 것으로 전해졌다.

빅토르 안은 2006년 토리노 동계올림픽에서 3개의 금메달을 딴 한국 쇼트트랙의 간판이었다. 2011년 소속팀이었던 성남시청이 빙상팀을 해체하자 선수 생활을 이어가기 위해 러시아로 귀화했다. 2014년 소치 동계올림픽에는 러시아 선수로 나서 3관왕에 올랐다. 2018년 평창 동계올림픽 출전이 무산된 이후에는 은퇴를 선언하고 지도자로 변신했다. 2022년 베이징 동계올림픽에서는 중국 대표팀 코치로 활동했다.

성남시는 31일 빙상팀 코치 선발 결과를 발표할 예정이다."""
embeded_content = embedder.encode([arc_content], convert_to_tensor=True)

In [219]:
%%time
cos_scores = util.pytorch_cos_sim(embeded_content, embeddings2)[0]

CPU times: total: 0 ns
Wall time: 5.53 ms


## ONNX 변환 모델

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sroberta-multitask")

In [129]:
%%time
embeddings_onnx_list = []
for sent in data_fit.iloc[:100,0]:
    model_inputs = tokenizer(sent, return_tensors='pt')
    inputs_onnx = {k: v.cpu().detach().numpy() for k , v in model_inputs.items()}
    try:
        embeddings_onnx_list.append(mean_pooling(embedder_onnx.run(None, inputs_onnx), inputs_onnx['attention_mask'])[0][0])
    except:
        pass
embeddings_onnx = torch.stack(embeddings_onnx_list, 0)

CPU times: total: 2min 13s
Wall time: 23.7 s


In [132]:
len(embeddings_onnx)

85

In [128]:
%%time
model_inputs = tokenizer('서울 신문 추미애 구치소 집단 감염 첫 사과 집단 감염 송구 취약 부분 드러나다 빠르다 후속 작업 진행 하다 동부 구치소 관련 확진 총 945 명 서 울 동부 구치소 코로나 19 신종 코로나바이러스 감염증 집단 감염 사태 관련 추미애 법무부 장관 처음 공식 사과 입장 밝히다 동부 구치소 관련 확 진자 발생 지난 11월 말 이후 하다 달이다 만이 추 장관 1일 자신 소셜네트워크서비스 계정 통해 신년인사 전하 동부 구치소 코로나 확산 대해 교정 업무 총괄 있다 법무부 장관 국민 심려 끼치다 드리다 매우 송구 하다 밝히다 법무 행정 취약 부분 드러나다 날 추 장관 코로나 같다 감염병 우리 사회 가장 취약 부분 먼저 무너 뜨다 린다 법무 행정 평소 취약 부분 드러나다 되다 대규모 감염병 사태 아주 치명 수 용소 과밀 그러하다 동부 구치소 지난 12월 25일 전문가 함께 점검 실시 하다 빠르다 집단 감염 원인 주로 3 차 대유행 후 무증상 감염 신입 수용 추정 돼다 말 하다 그렇다 이전 신입 수용 14일 간 격리 후 혼거 수용 하다 절차 준수 하다 하다 그러나 확 진자 증상 없다 걸다 지지 않다 이로 인하다 확산 가능성 여전하다 남아 있다 것 라며 추가 확산 방지 위해 확 진자 비 확 진자 분리 수용 밀도 낮추다 한다는 전문가 권고 받다 전 하다 아우르다 추 장관 동부 구치소 생활 치료 시설 지정 하다 이후 확 진자 수용 하다 시설 재 편하다 빠르다 시일 내 비확 진자 타 교정 기관 송해 분리 하다 계획 라며 또 모범 수형 대한 가석방 확대 형 집행정지 등 동시 진행 하다 빠르다 시 일 내 수용 밀도 낮추다 후속 작업 진행 하다 약속 하다 추 장관 구치소 교도소 달리 구속 또는 형 확정 되다 않다 미결 수 수용 하다 곳 신입 수용 자의 입감 및 출감 빈번 하다 교도소 달리 교정 당국 적정 인원 수용 등 조정 하다 수 있다 곳 아니다 항상 과밀 대한 우려 있다 하다 동부 구치소 감염병 매우 취약 구 조물 추 장관 더군다나 동부 구치소 고층 빌딩 형태 전형 3 밀 밀접 밀집 밀폐 구조 건물 간 간격 촘촘 가리개 설치 공기 흐름 막히다 환기 제대로 안 돼다 감염병 매우 취약 구조 물이 라며 향후 이러하다 부분 대한 개선 반드시 필요하다 것 지적 하다 추 장관 다시 한번 신년인사 전하 저 법무부 장관 임기 마지막 코로나 확산 방지 최선 다 하다 다시 한번 심려 끼치다 드리다 송구 하다 재차 사과 하다 법무부 2 주간 교정 시설 사회 거리 두기 3 단계 법무부 전날 2 주간 교정 시설 사회 거리 두기 3 단계 격상 하다 또 수감 되다 수용 자 불안 커지다 있다 점 고려 모든 교정 시설 직원 및 수용 대상 1 주일 1 이다 3 매 94 마스크 지급 하다 하다 이번 집단 감염 사태 관련 하다 법무부 대처 미흡하다 지적 이어지다 이용구 법무부 차관 당일 브리핑 통해 선제 방역 조치 미흡 이번 같다 사태 발생 하다 음 국민 여러분 송구 하다 말씀 드리다 사과 하다 다만 추 장관 브리핑 참석 하다 않다 추 장관 같다 날 오후 고층 빌딩 형태 인천 구치소 수원구치소 찾다 코로나 19 관련 주요 조치 사항 보고 받다 뒤 직원 및 수용 자의 전 수 검사 지시 하다 선제 방역 조치 중요성 강조 하다 하다 동부 구치소 관련 확 진자 총 945 명 1일 서울시 법무부 따르다 날 오전 0시 기준 동부 구치소 관련 확 진자 총 945 명 격리 추적 검사 과정 수용 131 명의 추가 감염 확인 돼다 교정 당국 따르다 날 오후 5시 기준 동부 구치소 수용 13 명과 직원 1 명 추가 확진 판정 받다 밝히다 확진 수용 13 명 최근 4 차 전 수 조사 결과 나오다 않다 미결 정자 14 명 중 일부 나머지 1 명 음성 판정 나오다 이 따르다 동부 구치소 관련 확 진자 수 수용 915 명 직원 22 명 등 모두 937 명 동부 구치소 지다 감염 지난해 11월 27일 송파구 거주 수능 수험생 최초 확진 판정 받다 이후 이 확 진자 가족 근무 하다 동부 구치소 동료 재소 가족 및 지인 등 급속하다 전파 돼다 하다 달이다 관련 확 진자 945 명 돼다 한편 법무부 공 무직 노동조합 12월 31일 동부 구치소 코로나 19 확 진자 계속 발생 사망자 발생 하다 추 장관 직무 유기 등 혐의 대검찰청 고발 하다 김채현 기자', return_tensors='pt')
inputs_onnx = {k: v.cpu().detach().numpy() for k , v in model_inputs.items()}
sequence = embedder_onnx.run(None, inputs_onnx)
print(sequence)
embeddings_onnx = mean_pooling(sequence, inputs_onnx['attention_mask'])[0][0]
embeddings_onnx

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Non-zero status code returned while running Gather node. Name:'/embeddings/position_embeddings/Gather' Status Message: indices element out of data bounds, idx=514 must be within the inclusive range [-514,513]

In [226]:
import numpy as np
import onnxruntime as ort
opt = ort.SessionOptions()
ort_session = ort.InferenceSession('./model/onnx/ko-sroberta-multitask.onnx', opt)

In [227]:
%%time
text = '서울 신문 추미애 구치소 집단 감염 첫 사과 집단 감염 송구 취약 부분 드러나다 빠르다 후속 작업 진행 하다 동부 구치소 관련 확진 총 945 명 서 울 동부 구치소 코로나 19 신종 코로나바이러스 감염증 집단 감염 사태 관련 추미애 법무부 장관 처음 공식 사과 입장 밝히다 동부 구치소 관련 확 진자 발생 지난 11월 말 이후 하다 달이다 만이 추 장관 1일 자신 소셜네트워크서비스 계정 통해 신년인사 전하 동부 구치소 코로나 확산 대해 교정 업무 총괄 있다 법무부 장관 국민 심려 끼치다 드리다 매우 송구 하다 밝히다 법무 행정 취약 부분 드러나다 날 추 장관 코로나 같다 감염병 우리 사회 가장 취약 부분 먼저 무너 뜨다 린다 법무 행정 평소 취약 부분 드러나다 되다 대규모 감염병 사태 아주 치명 수 용소 과밀 그러하다 동부 구치소 지난 12월 25일 전문가 함께 점검 실시 하다 빠르다 집단 감염 원인 주로 3 차 대유행 후 무증상 감염 신입 수용 추정 돼다 말 하다 그렇다 이전 신입 수용 14일 간 격리 후 혼거 수용 하다 절차 준수 하다 하다 그러나 확 진자 증상 없다 걸다 지지 않다 이로 인하다 확산 가능성 여전하다 남아 있다 것 라며 추가 확산 방지 위해 확 진자 비 확 진자 분리 수용 밀도 낮추다 한다는 전문가 권고 받다 전 하다 아우르다 추 장관 동부 구치소 생활 치료 시설 지정 하다 이후 확 진자 수용 하다 시설 재 편하다 빠르다 시일 내 비확 진자 타 교정 기관 송해 분리 하다 계획 라며 또 모범 수형 대한 가석방 확대 형 집행정지 등 동시 진행 하다 빠르다 시 일 내 수용 밀도 낮추다 후속 작업 진행 하다 약속 하다 추 장관 구치소 교도소 달리 구속 또는 형 확정 되다 않다 미결 수 수용 하다 곳 신입 수용 자의 입감 및 출감 빈번 하다 교도소 달리 교정 당국 적정 인원 수용 등 조정 하다 수 있다 곳 아니다 항상 과밀 대한 우려 있다 하다 동부 구치소 감염병 매우 취약 구 조물 추 장관 더군다나 동부 구치소 고층 빌딩 형태 전형 3 밀 밀접 밀집 밀폐 구조 건물 간 간격 촘촘 가리개 설치 공기 흐름 막히다 환기 제대로 안 돼다 감염병 매우 취약 구조 물이 라며 향후 이러하다 부분 대한 개선 반드시 필요하다 것 지적 하다 추 장관 다시 한번 신년인사 전하 저 법무부 장관 임기 마지막 코로나 확산 방지 최선 다 하다 다시 한번 심려 끼치다 드리다 송구 하다 재차 사과 하다 법무부 2 주간 교정 시설 사회 거리 두기 3 단계 법무부 전날 2 주간 교정 시설 사회 거리 두기 3 단계 격상 하다 또 수감 되다 수용 자 불안 커지다 있다 점 고려 모든 교정 시설 직원 및 수용 대상 1 주일 1 이다 3 매 94 마스크 지급 하다 하다 이번 집단 감염 사태 관련 하다 법무부 대처 미흡하다 지적 이어지다 이용구 법무부 차관 당일 브리핑 통해 선제 방역 조치 미흡 이번 같다 사태 발생 하다 음 국민 여러분 송구 하다 말씀 드리다 사과 하다 다만 추 장관 브리핑 참석 하다 않다 추 장관 같다 날 오후 고층 빌딩 형태 인천 구치소 수원구치소 찾다 코로나 19 관련 주요 조치 사항 보고 받다 뒤 직원 및 수용 자의 전 수 검사 지시 하다 선제 방역 조치 중요성 강조 하다 하다 동부 구치소 관련 확 진자 총 945 명 1일 서울시 법무부 따르다 날 오전 0시 기준 동부 구치소 관련 확 진자 총 945 명 격리 추적 검사 과정 수용 131 명의 추가 감염 확인 돼다 교정 당국 따르다 날 오후 5시 기준 동부 구치소 수용 13 명과 직원 1 명 추가 확진 판정 받다 밝히다 확진 수용 13 명 최근 4 차 전 수 조사 결과 나오다 않다 미결 정자 14 명 중 일부 나머지 1 명 음성 판정 나오다 이 따르다 동부 구치소 관련 확 진자 수 수용 915 명 직원 22 명 등 모두 937 명 동부 구치소 지다 감염 지난해 11월 27일 송파구 거주 수능 수험생 최초 확진 판정 받다 이후 이 확 진자 가족 근무 하다 동부 구치소 동료 재소 가족 및 지인 등 급속하다 전파 돼다 하다 달이다 관련 확 진자 945 명 돼다 한편 법무부 공 무직 노동조합 12월 31일 동부 구치소 코로나 19 확 진자 계속 발생 사망자 발생 하다 추 장관 직무 유기 등 혐의 대검찰청 고발 하다 김채현 기자'
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input = {name : np.atleast_2d(value) for name, value in encoded_input.items()}

print(ort_session.run(None, encoded_input)[0])

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Non-zero status code returned while running Gather node. Name:'/embeddings/position_embeddings/Gather' Status Message: indices element out of data bounds, idx=514 must be within the inclusive range [-514,513]

## ONNX + 양자화 모델

In [135]:
%%time
embeddings_onnx_uint8_list = []
for sent in data_fit.iloc[:100,0]:
    model_inputs = tokenizer(sent, return_tensors='pt')
    inputs_onnx = {k: v.cpu().detach().numpy() for k , v in model_inputs.items()}
    embeddings_onnx_list.append(mean_pooling(embedder_onnx_uint8.run(None, inputs_onnx), inputs_onnx['attention_mask'])[0][0])
embeddings_onnx_uint8 = torch.stack(embeddings_onnx_uint8_list, 0)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Non-zero status code returned while running Gather node. Name:'/embeddings/position_embeddings/Gather' Status Message: indices element out of data bounds, idx=514 must be within the inclusive range [-514,513]